# Z3 rešavač

In [3]:
from z3 import *

In [4]:
from IPython.display import display

## Logičke promenljive

Za definisanje logičkih promenljivih koristimo funkciju *Bools* koja nam vraća torku referenci ka nevezanim promelnjivima sa prosleđenim imenima.

Funkciom *solve* prosleđujemo ograničenja koja želimo da važe, a z3 nam vraća jedno moguće rešenje, ako ono postoji.

**Zadatak**: U iskaznoj logici zapisati uslov da je četvorobitni broj palindrom, ali da nisu svi bitovi isti.

In [11]:
A, B, C, D = Bools("A B C D")
solve(A == D, B == C, Not(And(A == B, B == C, C == D)))

[A = True, D = True, B = False, C = False]
1


## Realne promenljive

Realne nevezane promenljive definišemo funckijom *Reals*, analogno logičkim promenljivima.

**Zadatak:** Implementirati rešavač linearnih jednačina sa tri nepoznate.

In [18]:
x, y, z = Reals('x y z')

solve(
    x + 5*y - 3*z == 4,
    -x + y + z == 3,
    2*x + y - z == 1
)

[y = 7/4, x = 1/2, z = 7/4]


## Funkcije

U z3 možemo da definišemo funkcijske promenljive, a zatim za zadata ograničenja rešavač može da nađe interpretaciju za date funkcije.

Za definiciju funkcijskih promenljivih, koristi se funckija *Function*. Njoj se prosleđuje ime funkcijske promenljive, kao i njen potpis.

*IntSort* predstavlja domen celih brojeva, a *BoolSort* domen logičkih vrednosti.

Funkcijom *BoolVal* se definiše interpretirana konstanta logičkog tipa.

**Zadatak:** Naći funkcie $ f : \mathbb{Z} \rightarrow \mathbb{B} $ i $g : \mathbb{B} \rightarrow \mathbb{Z}$ takve da važi ograničenje $ g(1 + f(a)) = True $.

In [19]:
f = Function('f', BoolSort(), IntSort())
g = Function('g', IntSort(), BoolSort())

a = Bool('a')
t = BoolVal(True)

solve(g(1 + f(a)) == t)

[a = False, f = [else -> 0], g = [else -> True]]


## Korisnički definisani domeni

In [23]:
T = DeclareSort("Tacke")
t1, t2 = Consts("t1 t2", T)

## Razni zadaci

**Zadatak:** Za zadate aksiome, proveriti da li važi konjektura.

Aksiome:
- Dve nemimoilzne prave se seku ili su parelelne-.
- Prave koje se seku leze u istoj ravni.
- Prave koje su paralelne leze u istoj ravni.

Konjektura: Dve nemimoilazne prave leze u istoj ravni.

In [29]:
# m(X, Y) - X i Y su nemimoilazne prave: m: PxP -> B
# s(X, Y) - X i Y se: s: PxP -> B
# p(X, Y) - X i Y su parelene: s: PxP -> B
# r(X, Y) - X i Y leze u istoj ravni: s: PxP -> B

B = BoolSort()
P = DeclareSort("Prave")

m = Function("m", P, P, B)
s = Function("s", P, P, B)
p = Function("p", P, P, B)
r = Function("r", P, P, B)

x, y = Consts("x y", P)

axioms = [
    ForAll([x,y], Implies(m(x,y), Or(s(x,y), p(x,y)))),
    ForAll([x,y], Implies(s(x,y), r(x,y))),
    ForAll([x,y], Implies(p(x,y), r(x,y)))
]
conjectures = [ForAll([x,y], Implies(m(x,y), r(x,y)))]

solver = Solver()
solver.add(axioms)

print(solver.check(conjectures))

sat


U ovom primeru nam nije bitan model, već samo da li je zadati skup ograničenja zadovoljiv ili nije.

**Zadatak:** N dama

$ Q_i $ - na kom polju u $i$-toj vrsti je postavljena dama.

```value_constraint``` - kolone su označene brojevima od $0$ do $n$.

```column_constraint``` - svaka dama mora da stoji u različitoj koloni.

```diag_constraint``` - dame se ne mogu napadati po dijagonalama: $|Q_i - Q_j| \ne |i - j|$

In [30]:
n = 8
Q = [Int(f"Q_{i}") for i in range(n)]

value_constraint = [And(0 <= q, q < n) for q in Q]
column_constraint = [Distinct(Q)]

diag_constraint = [
    And(Q[i] - Q[j] != i - j, Q[i] - Q[j] != j - i)
    for i in range(n) for j in range(i) if i != j
]

n_queens = val_c + col_c + diag_c
solve(n_queens)

[Q_3 = 2,
 Q_1 = 6,
 Q_7 = 1,
 Q_5 = 5,
 Q_4 = 0,
 Q_0 = 3,
 Q_2 = 4,
 Q_6 = 7]


In [19]:
# Za vezbu implementirati probleme sa 7. casa